In [1]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Resizing, Rescaling, Dense, Conv2D, MaxPooling2D, Flatten, RandomFlip, RandomContrast, RandomRotation
from tensorflow.keras import Sequential
from tensorflow.image import random_flip_up_down, random_flip_left_right, random_contrast, random_brightness, rot90
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import random
import numpy as np
from tensorflow.keras.utils import image_dataset_from_directory

In [2]:
# Set seed for reproduceability
SEED = 42

# Size to convert images to (pixels)
RESIZE_HEIGHT = 750
RESIZE_WIDTH = 750

In [27]:
# Load photo data
train_images = image_dataset_from_directory(directory = 'data/Training', color_mode="grayscale", label_mode = 'binary', batch_size=32)
test_images = image_dataset_from_directory(directory = 'data/Testing', color_mode="grayscale", label_mode = 'binary', batch_size=32)

Found 5712 files belonging to 2 classes.
Found 1311 files belonging to 2 classes.


In [29]:
# Images are not standardized. Need to standardize prior to modelling
def resize_images(image, label):
    image = tf.image.resize_with_crop_or_pad(image, target_height = RESIZE_HEIGHT, target_width = RESIZE_WIDTH)
    return image, label

train_images = train_images.map(resize_images)
test_images = test_images.map(resize_images)

In [30]:
# Runtime optimization
AUTOTUNE = tf.data.AUTOTUNE 
train_images = train_images.prefetch(buffer_size = AUTOTUNE)
test_images = test_images.prefetch(buffer_size = AUTOTUNE)

In [31]:
# Create initial photo standardization preprocessing segment.
# Input shape = (height, width, 1 = greyscale)
standardization_layer = Sequential([
    #Resizing(RESIZE_WIDTH, RESIZE_HEIGHT),
    Rescaling(1 / 255.)
])

In [32]:
for images, labels in train_images.take(1):
    print(images.shape, labels.shape)


(32, 750, 750, 1) (32, 1)


In [33]:
# Create an augmentation layer that randomly rotates, flips, and contrasts the images.
augmentation_layer = Sequential([
    tf.keras.Input(shape = (RESIZE_HEIGHT, RESIZE_WIDTH, 1)),
    RandomRotation(factor = (-0.3, 0.3)),
    RandomFlip("horizontal_and_vertical"),
    RandomContrast(factor = (-0.3, 0.3))
])

In [34]:
# Base model with combination of preprocessing layers and convolutional/pooling/dense layers
model = Sequential([
    standardization_layer,
    augmentation_layer,
    Conv2D(16, (3,3), activation = 'relu'),
    MaxPooling2D(),
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [38]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy','recall'])

model.fit(train_images, validation_data=test_images, epochs=10, batch_size=256)

Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.9589 - loss: 0.1266 - recall: 0.9746 - val_accuracy: 0.9344 - val_loss: 0.1788 - val_recall: 0.9614
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.9644 - loss: 0.1104 - recall: 0.9790 - val_accuracy: 0.9375 - val_loss: 0.1634 - val_recall: 0.9349
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.9623 - loss: 0.1136 - recall: 0.9755 - val_accuracy: 0.9542 - val_loss: 0.1587 - val_recall: 0.9558
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.9653 - loss: 0.1064 - recall: 0.9804 - val_accuracy: 0.9252 - val_loss: 0.2006 - val_recall: 0.9625
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 196s 1s/step - accuracy: 0.9679 - loss: 0.0995 - recall: 0.9826 - val_accuracy: 0.9428 - val_loss: 0.1608 - val_recall: 0.9558
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.9702 - loss: 0.0966 - recall: 0.9813 - val_accuracy: 0.9405 - val_loss: 0.1485 - val_recall: 0.958